 На языке Python разработайте скрипт, который с помощью генетического алгоритма решает следующую задачу. Дано n пунктов производства продуктов и k городов, которые в них нуждаются. Каждый город может потребить x продуктов, а каждый пункт произвести Y продуктов. Необходимо получить оптимальный маршрут, так, чтобы все города получили нужный им объем продуктов с минимальным его превышением, а транспортные расходы укладывались в определенные рамки.

In [138]:
import random
import numpy as np

In [139]:
n = 3 
k = 5 

In [140]:
print(f"Количество пунктов производства : {n}, Количество городов : {k}")

Количество пунктов производства : 3, Количество городов : 5


In [141]:
X = [20, 30, 40, 50, 60]
Y = [100, 60, 100]
distance = [[10, 15, 20, 10, 30],[20, 15, 10, 20, 10],[30, 10, 20, 15, 10]]
MAX_COST = 200 #ограничение

In [142]:
for i in range(k):
    print(f"Потребление городом {i+1} продуктов : {X[i]}")
for i in range(n):
    print(f"Произведение пунктом {i+1} продуктов : {Y[i]}")
for i in range(n):
    for j in range(k):
        print("\t"*i+f"Расстояние между пунктом {i+1} и городом {j+1} = {distance[i][j]}")

Потребление городом 1 продуктов : 20
Потребление городом 2 продуктов : 30
Потребление городом 3 продуктов : 40
Потребление городом 4 продуктов : 50
Потребление городом 5 продуктов : 60
Произведение пунктом 1 продуктов : 100
Произведение пунктом 2 продуктов : 60
Произведение пунктом 3 продуктов : 100
Расстояние между пунктом 1 и городом 1 = 10
Расстояние между пунктом 1 и городом 2 = 15
Расстояние между пунктом 1 и городом 3 = 20
Расстояние между пунктом 1 и городом 4 = 10
Расстояние между пунктом 1 и городом 5 = 30
	Расстояние между пунктом 2 и городом 1 = 20
	Расстояние между пунктом 2 и городом 2 = 15
	Расстояние между пунктом 2 и городом 3 = 10
	Расстояние между пунктом 2 и городом 4 = 20
	Расстояние между пунктом 2 и городом 5 = 10
		Расстояние между пунктом 3 и городом 1 = 30
		Расстояние между пунктом 3 и городом 2 = 10
		Расстояние между пунктом 3 и городом 3 = 20
		Расстояние между пунктом 3 и городом 4 = 15
		Расстояние между пунктом 3 и городом 5 = 10


In [143]:
#создаём популяцию
def generate_population(size):
    return [[random.randint(0, n - 1) for i in range(k)] for i in range(size)]

In [144]:
print(f"{generate_population(10)}")

[[1, 2, 1, 2, 1], [1, 2, 0, 1, 0], [0, 1, 0, 2, 0], [1, 1, 1, 1, 1], [0, 1, 0, 2, 2], [0, 0, 1, 1, 2], [2, 0, 2, 0, 1], [0, 1, 1, 0, 0], [0, 0, 2, 2, 2], [2, 1, 0, 2, 1]]


In [145]:
#приспособленность
def fitness(individual):
    total_supply = [0] * n
    total_cost = 0
    for city, factory in enumerate(individual):
        total_supply[factory] += X[city]
        total_cost += distance_matrix[factory][city]
    #штраф
    supply_excess = sum(max(0, total_supply[i] - Y[i]) for i in range(n))
    
    unmet_demand = sum(max(0, city_demand[city] - total_supply[factory]) for city, factory in enumerate(individual))
    
    # Если транспортные расходы превышают лимит, добавляем большой штраф
    if total_cost > MAX_COST:
        return float('inf')
    
    return total_cost + supply_excess + unmet_demand
    return total_cost + supply_excess

In [146]:
#скрещивание
def crossover(parent1,parent2):
    point = random.randint(1,k-1)
    child1 = parent1[:point] +parent2[point:]
    child2 = parent2[:point]+parent1[point:]
    return child1,child2

In [147]:
#мутация
def mutate(individual):
    point = random.randint(0, k - 1)
    individual[point] = random.randint(0, n - 1)
    return individual

In [148]:
#селекция
def selection(population, fitnesses):
    selected = random.choices(population, weights=[1/f for f in fitnesses], k=1)
    return selected[0]

In [149]:
#Основа кода
def genetic_algorithm(pop_size, generations, cx_prob, mut_prob):
    population = generate_population(pop_size)
    
    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]
        new_population = []
        #выбираем, мутируем и создаём популяцию
        for _ in range(pop_size // 2):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            if random.random() < cx_prob:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1, parent2
            if random.random() < mut_prob:
                child1 = mutate(child1)
            if random.random() < mut_prob:
                child2 = mutate(child2)
            
            new_population.extend([child1, child2])
        population = new_population
    
    #лучший индивид
    final_fitnesses = [fitness(ind) for ind in population]
    best_index = np.argmin(final_fitnesses)
    return population[best_index], final_fitnesses[best_index]

In [150]:
if __name__ == "__main__":
    best_individual, best_fitness = genetic_algorithm(pop_size=50, generations=10, cx_prob=0.7, mut_prob=0.1)
    print(f"Оптимальный маршрут с затратами : {best_fitness}")
    for k, n in enumerate(best_individual):
        print(f"Город {k + 1} получает продукты из пункта {n + 1}")

Оптимальный маршрут с затратами : 50
Город 1 получает продукты из пункта 1
Город 2 получает продукты из пункта 3
Город 3 получает продукты из пункта 2
Город 4 получает продукты из пункта 1
Город 5 получает продукты из пункта 3
